<div class="alert alert-block alert-info" style="background-color: #3066BE; color: white;">
    <h1 style="color: white;">DSA3101 Backend</h1>
</div>

In [35]:
# !pip install langchain
# !pip install --upgrade --quiet  langchain-google-vertexai

In [19]:
# import dotenv
# dotenv.load_dotenv()

In [10]:
# pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Setting up google-auth 
- https://mayankgpt50.medium.com/setting-up-gcp-account-with-local-system-unlocking-bigquery-and-cloud-storage-using-jupyter-8b13435118c4

In [1]:
# !pip install google-auth
# !pip install google-cloud-bigquery
# !pip install google-cloud-storage

In [6]:
from google.cloud import bigquery
import google.auth

# Create credentials with Drive & BigQuery API scopes.
# Both APIs must be enabled for your project before running this code.
#
# If you are using credentials from gcloud, you must authorize the
# application first with the following command:
#
# gcloud auth application-default login \
#   --scopes=https://www.googleapis.com/auth/drive,https://www.googleapis.com/auth/cloud-platform

credentials, project = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/drive",
        "https://www.googleapis.com/auth/cloud-platform",
    ]
)

# Construct a BigQuery client object.
client = bigquery.Client(credentials=credentials, project=project)

In [7]:
# Importing the library 
import google.auth

# Setting up credentials
credentials, _ = google.auth.default()
credentials = google.auth.credentials.with_scopes_if_required(credentials, bigquery.Client.SCOPE)
authed_http = google.auth.transport.requests.AuthorizedSession(credentials)

# Code begins here

In [1]:
import pandas as pd

from langchain.prompts import ChatPromptTemplate
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai import VertexAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

ModuleNotFoundError: No module named 'langchain_google_vertexai'

## 1. Retrieving relevant product features

- Consumers to key in types of product that they bought for more active engagement
- Comments: Instead of keying in, have dropdown fields for consumers to select to streamline information

- Once selection is made, a call is made from FRONT to BACK... (eg. >>> Fabric Softener)

In [3]:
# df = pd.read_csv('product_information.csv')
df = pd.read_csv('product_info.csv')
df

,Product,Features
0,Diapers,"Price, Comfort, Absorbance, Fit, Material"
1,Pants,"Price, Comfort, Absorbance, Fit, Material"
2,Wipes,"Price, Fragrance, Packaging, Quality"
3,Fabric Softener,"Price, Scent, Packaging, Utility, Effectiveness"
4,Scent Beads,"Price, Scent, Packaging, Effectiveness"


In [4]:
# input from frontend
selected_product = "Fabric Softener"

features = df.loc[df['Product'] == selected_product, 'Features'].values[0]
features

'Price, Scent, Packaging, Utility, Effectiveness'

In [5]:
# input from frontend
feedback = "The fabric softener has a nice floral scent, have been using this softener for quite some time. The price is okay too since I bought it as a discounted value pack."

## 2. Using LLM to extract response on product features

In [50]:
system = """
Your job is to extract different features of a product from customer feedbacks. 

Consider the following format for output, leave response as a python dictionary:
# Feature 1: Concise review on feature 1
# Feature 2: Concise review on feature 2
# Feature 3: Concise review on feature 3
# Feature 4: Concise review on feature 4

Eg. 
Price: Reasonable
Scent: Pleasant and refreshing smell
Effectiveness: Good

Here is the list of features to categorise the feedback into: {features}
If features are not mentioned, leave section as empty python string.
Provide your output here:
"""

human = """
{feedback}
"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chat = ChatVertexAI(model_name="gemini-pro", convert_system_message_to_human=True)
# chat = ChatVertexAI(llm=llm, convert_system_message_to_human=True)

chain = prompt | chat
text = chain.invoke({
    "feedback": feedback,
    "features": features,
})

In [51]:
text

AIMessage(content='{\n"Price": "The price is okay.",\n"Scent": "The fabric softener has a nice floral scent",\n"Packaging": "",\n"Utility": "",\n"Effectiveness": ""\n}', response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}], 'citation_metadata': None, 'usage_metadata': {'prompt_token_count': 178, 'candidates_token_count': 42, 'total_token_count': 220}})

In [52]:
text.content

'{\n"Price": "The price is okay.",\n"Scent": "The fabric softener has a nice floral scent",\n"Packaging": "",\n"Utility": "",\n"Effectiveness": ""\n}'

In [53]:
features_dict = eval(text.content)
features_dict

{'Price': 'The price is okay.',
 'Scent': 'The fabric softener has a nice floral scent',
 'Packaging': '',
 'Utility': '',
 'Effectiveness': ''}

### Identify missing features

In [30]:
missing_features = []
for f in features_dict:
    if features_dict[f] == "":
        missing_features.append(f)
missing_features

['Packaging', 'Utility', 'Effectiveness']

## 3. Generate new questions based on missing features

In [61]:
template = """
Your job is to ask customers questions about the {selected_product} based on the {missing_features}.
"""

prompt = PromptTemplate(template=template, input_variables=["selected_product", "missing_feature"])
llm_chain = LLMChain(llm=llm, prompt=prompt)
questions = llm_chain.invoke(input={'selected_product':selected_product, 'missing_features':missing_features})

In [64]:
questions['text']

'**Packaging**\n\n* How satisfied are you with the packaging of the fabric softener?\n* Is the packaging easy to open and reseal?\n* Do you find the packaging to be attractive and informative?\n* Are there any improvements you would suggest for the packaging?\n\n**Utility**\n\n* How easy is it to use the fabric softener?\n* Does the fabric softener dispense easily from the bottle?\n* Can you use the fabric softener in both top-loading and front-loading washing machines?\n* Are there any specific instructions or precautions you need to follow when using the fabric softener?\n\n**Effectiveness**\n\n* How effective is the fabric softener at softening your clothes?\n* Does the fabric softener reduce static cling?\n* Does the fabric softener leave a pleasant scent on your clothes?\n* Have you noticed any improvement in the texture or appearance of your clothes after using the fabric softener?\n* Are there any specific types of fabrics or garments that you find the fabric softener to be part